In [1]:
%cd ..

/home/akajal/DiVAE


In [1]:
# Python imports
import os
import sys

# Add the path to the parent directory to augment search for module
sys.path.append(os.getcwd())
# Add the path to the parent directory to augment search for module
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if par_dir not in sys.path:
    sys.path.append(par_dir)
    
# ML imports
import torch
import matplotlib.pyplot as plt
import numpy as np
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

# DiVAE imports
from models.rbm.chimeraRBM import ChimeraRBM
from models.rbm.qimeraRBM import QimeraRBM
from models.rbm.rbm import RBM
from models.samplers.pcd import PCD
from models.autoencoders.gumboltCaloCRBM import GumBoltCaloCRBM

from nbutils import *

# DWave imports
from dwave.system import DWaveSampler, LeapHybridSampler
import neal

ModuleNotFoundError: No module named 'DiVAE'

In [3]:
def load_state(model, run_path, device):
        model_loc = run_path
        
        # Open a file in read-binary mode
        with open(model_loc, 'rb') as f:
            # Interpret the file using torch.load()
            checkpoint=torch.load(f, map_location=device)
            
            local_module_keys=list(model._modules.keys())
            for module in checkpoint.keys():
                if module in local_module_keys:
                    print("Loading weights for module = ", module)
                    getattr(model, module).load_state_dict(checkpoint[module])

In [4]:
def init_model(cfg):
    model = GumBoltCaloCRBM(flat_input_size=[504],
                            train_ds_mean=0.,
                            activation_fct=torch.nn.ReLU(),
                            cfg=cfg)
    return model

In [5]:
with initialize(config_path="../configs"):
    cfg = compose(config_name="config")

/opt/conda/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [6]:
print(cfg)

{'model': {'model_type': 'GumBoltCaloCRBM', 'activation_fct': 'relu', 'beta_smoothing_fct': 5, 'output_smoothing_fct': 9, 'decoder_hidden_nodes': [800, 700, 600, 550, 500], 'encoder_hidden_nodes': [500, 450, 400, 350, 300], 'n_latent_hierarchy_lvls': 2, 'n_latent_nodes': 1000, 'n_encoder_layer_nodes': -1, 'n_encoder_layers': -1, 'mapping': 'bipartite', 'bernoulli': 0}, 'data': {'data_type': 'calo', 'calo_layers': ['layer_0', 'layer_1', 'layer_2'], 'particle_type': 'piplus', 'frac_test_dataset': 0.1, 'frac_train_dataset': 0.8, 'scaled': True, 'scaler_path': '/fast_scratch/QVAE/data/calo_scaled/piplus_scaler.gz', 'scaler_amin': '/fast_scratch/QVAE/data/calo_scaled/piplus_amin.npy', 'calo_input_eplus': '/fast_scratch/QVAE/data/calo_scaled/eplus.hdf5', 'calo_input_gamma': '/fast_scratch/QVAE/data/calo_scaled/gamma.hdf5', 'calo_input_piplus': '/fast_scratch/QVAE/data/calo_scaled/piplus.hdf5'}, 'engine': {'_target_': 'engine.engineCaloV3.EngineCaloV3', 'learning_rate': 0.0001, 'n_train_batch

In [7]:
model = init_model(cfg)
model.create_networks()

2022-03-28 23:54:08,711 dwave.cloud.client.base INFO  MainThread Fetching available regions from the Metadata API at 'https://cloud.dwavesys.com/metadata/v1/'
[23:54:08.711] INFO   dwave.cloud.client.base                           Fetching available regions from the Metadata API at 'https://cloud.dwavesys.com/metadata/v1/'
2022-03-28 23:54:08,990 dwave.cloud.client.base INFO  MainThread Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
[23:54:08.990] INFO   dwave.cloud.client.base                           Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
2022-03-28 23:54:09,035 dwave.cloud.client.base INFO  MainThread Req

Initializing NetworkV3


In [8]:
type(model.prior)

models.rbm.qimeraRBM.QimeraRBM

In [9]:
run_path = '/home/akajal/DiVAE/outputs/2022-01-24/14-14-25/wandb/run-20220124_141425-3o4vumug/files/GumBoltCaloCRBM_calo_default_latest.pth'
load_state(model, run_path, 'cuda:{0}'.format(cfg.gpu_list[0]))

Loading weights for module =  _activation_fct
Loading weights for module =  _bce_loss
Loading weights for module =  _energy_activation_fct
Loading weights for module =  _hit_activation_fct
Loading weights for module =  _output_loss
Loading weights for module =  _hit_loss
Loading weights for module =  _hit_smoothing_dist_mod
Loading weights for module =  encoder
Loading weights for module =  prior
Loading weights for module =  decoder
Loading weights for module =  sampler


In [ ]:
model.prior.weights.size()

In [ ]:
model.prior.weights[model.prior.weights != 0].size()

In [ ]:
model.prior.visible_qubit_idxs[:10]

In [ ]:
def rbm_weights(model):
    weights = model.prior.weights.detach().cpu().numpy()
    weights = weights[np.nonzero(weights)]
    
    fig, ax = plt.subplots(figsize=(40, 16))
    
    plt.hist(weights, bins=100, alpha=0.5, edgecolor='black', linewidth=1.3)
    
    ax.set_xlabel("Weight", fontsize=60)
    ax.set_ylabel("Frequency", fontsize=60)
    
    ax.tick_params(axis='both', which='major', labelsize=60)
    ax.set_yscale("log")
    
    plt.show()
    plt.close()

In [ ]:
def rbm_biases(model):
    vis_bias = model.prior.visible_bias.detach().cpu().numpy()
    hid_bias = model.prior.hidden_bias.detach().cpu().numpy()
    biases = np.concatenate((vis_bias, hid_bias))
    
    fig, ax = plt.subplots(figsize=(40, 16))
    
    plt.hist(biases, bins=100, alpha=0.5, edgecolor='black', linewidth=1.3)
    
    ax.set_xlabel("Bias", fontsize=60)
    ax.set_ylabel("Frequency", fontsize=60)
    
    ax.tick_params(axis='both', which='major', labelsize=60)
    ax.set_yscale("log")
    
    plt.show()
    plt.close()

In [ ]:
rbm_weights(model)

In [ ]:
rbm_biases(model)

In [13]:
def ising_weights(model):
    weights = model.prior.weights.detach().cpu().numpy()
    weights = weights[np.nonzero(weights)]/4.
    
    fig, ax = plt.subplots(figsize=(40, 16))
    
    plt.hist(weights, bins=100, alpha=0.5, edgecolor='black', linewidth=1.3)
    
    ax.set_xlabel("Weight", fontsize=60)
    ax.set_ylabel("Frequency", fontsize=60)
    
    ax.tick_params(axis='both', which='major', labelsize=60)
    ax.set_yscale("log")
    
    plt.show()
    plt.close()

In [14]:
ising_weights(model)

RuntimeError: Numpy is not available

In [15]:
def ising_biases(model):
    rbm_weights = model.prior.weights
    vis_bias = model.prior.visible_bias
    hid_bias = model.prior.hidden_bias
    
    ising_vbias = vis_bias/2. + torch.sum(rbm_weights, dim=1)/4.
    ising_hbias = hid_bias/2. + torch.sum(rbm_weights, dim=0)/4.
    
    ising_vbias = ising_vbias.detach().cpu().numpy()
    ising_hbias = ising_hbias.detach().cpu().numpy()
    
    biases = np.concatenate((ising_vbias, ising_hbias))
    
    fig, ax = plt.subplots(figsize=(40, 16))
    
    plt.hist(biases, bins=100, alpha=0.5, edgecolor='black', linewidth=1.3)
    
    ax.set_xlabel("Bias", fontsize=60)
    ax.set_ylabel("Frequency", fontsize=60)
    
    ax.tick_params(axis='both', which='major', labelsize=60)
    ax.set_yscale("log")
    
    plt.show()
    plt.close()

In [16]:
ising_biases(model)

RuntimeError: Numpy is not available

Trained RBM on the QPU

In [ ]:
ZERO = torch.tensor(0., dtype=torch.float)
MINUS_ONE = torch.tensor(-1., dtype=torch.float)

In [ ]:
crbm_weights = model.prior.weights
crbm_vbias = model.prior.visible_bias
crbm_hbias = model.prior.hidden_bias

In [ ]:
ising_weights = crbm_weights/4.
ising_vbias = crbm_vbias/2. + torch.sum(crbm_weights, dim=1)/4.
ising_hbias = crbm_hbias/2. + torch.sum(crbm_weights, dim=0)/4.

In [ ]:
def beta_estimation(num_iterations=10, lr=0.1, beta_init=5., n_reads=100, crbm_sampler=None, dwave_sampler=None):
    assert crbm_sampler is not None
    assert dwave_sampler is not None
    
    beta = beta_init
    betas = [beta]
    
    crbm = crbm_sampler.get_rbm()
    crbm_weights, crbm_vbias, crbm_hbias = crbm.weights, crbm.visible_bias, crbm.hidden_bias
    crbm_edgelist = crbm.pruned_edge_list
    
    ising_weights = crbm_weights/4.
    ising_vbias = crbm_vbias/2. + torch.sum(crbm_weights, dim=1)/4.
    ising_hbias = crbm_hbias/2. + torch.sum(crbm_weights, dim=0)/4.
    
    n_vis = len(crbm.visible_qubit_idxs)
    qubit_idxs = crbm.visible_qubit_idxs+crbm.hidden_qubit_idxs
    
    visible_idx_map = {visible_qubit_idx:i for i, visible_qubit_idx in enumerate(crbm.visible_qubit_idxs)}
    hidden_idx_map = {hidden_qubit_idx:i for i, hidden_qubit_idx in enumerate(crbm.hidden_qubit_idxs)}
    
    for i in range(num_iterations):
        dwave_weights, dwave_vbias, dwave_hbias = -ising_weights/beta, -ising_vbias/beta, -ising_hbias/beta
        dwave_weights_np = dwave_weights.detach().cpu().numpy()
        biases = torch.cat((dwave_vbias, dwave_hbias)).detach().cpu().numpy()
        
        print("Iteration = {0}, beta = {1} : J range = ({2}, {3}), h range = ({4}, {5})".format(i, beta, np.min(dwave_weights_np),
                                                                                                np.max(dwave_weights_np), np.min(biases),
                                                                                                np.max(biases)))
        
        h = {qubit_idx:bias for qubit_idx, bias in zip(qubit_idxs, biases)}
        J = {}
        for edge in crbm_edgelist:
            if edge[0] in crbm.visible_qubit_idxs:
                J[edge] = dwave_weights_np[visible_idx_map[edge[0]]][hidden_idx_map[edge[1]]]
            else:
                J[edge] = dwave_weights_np[visible_idx_map[edge[1]]][hidden_idx_map[edge[0]]]
                    
        response = dwave_sampler.sample_ising(h, J, num_reads=n_reads, auto_scale=False)
        dwave_samples, dwave_energies = batch_dwave_samples(response)
        dwave_energy_exp = np.mean(dwave_energies)
        
        crbm_vis, crbm_hid = crbm_sampler.block_gibbs_sampling()
        
        crbm_vis = torch.where(crbm_vis == ZERO, MINUS_ONE, crbm_vis)
        crbm_hid = torch.where(crbm_hid == ZERO, MINUS_ONE, crbm_hid)
        
        crbm_energy_exp = sample_energies_qpu(dwave_weights, dwave_vbias, dwave_hbias, crbm_vis, crbm_hid)
        crbm_energy_exp = torch.mean(crbm_energy_exp, axis=0)
        
        print("crbm_energy_exp : {0}, dwave_energy_exp : {1}".format(crbm_energy_exp, dwave_energy_exp))
        beta = beta + lr*(float(crbm_energy_exp)-float(dwave_energy_exp))
        betas.append(beta)
        
    return betas

In [ ]:
_NUM_NODES = 16
crbm = ChimeraRBM(n_visible=_NUM_NODES, n_hidden=_NUM_NODES)
crbm_sampler = PCD(batch_size=1024, RBM=crbm, n_gibbs_sampling_steps=100)

qpu_sampler = DWaveSampler(solver={"topology__type":"chimera"})

beta_estimation(crbm_sampler=model.sampler, dwave_sampler=qpu_sampler)

In [ ]:
qpu_sampler = DWaveSampler(solver={"topology__type":"chimera", "chip_id":"DW_2000Q_6"})
betas = beta_estimation(num_iterations=20, beta_init=10., lr=0.01, crbm_sampler=model.sampler, dwave_sampler=qpu_sampler)

In [ ]:
plot_betas(betas)

In [ ]:
def energy_deltas_ising(beta_min=1, beta_max=100, n_reads=100, crbm_sampler=None, dwave_sampler=None):
    assert crbm_sampler is not None
    assert dwave_sampler is not None
    
    crbm = crbm_sampler.get_rbm()
    crbm_weights, crbm_vbias, crbm_hbias = crbm.weights, crbm.visible_bias, crbm.hidden_bias
    crbm_edgelist = crbm.pruned_edge_list
    
    ising_weights = crbm_weights/4.
    ising_vbias = crbm_vbias/2. + torch.sum(crbm_weights, dim=1)/4.
    ising_hbias = crbm_hbias/2. + torch.sum(crbm_weights, dim=0)/4.
    
    n_vis = len(crbm.visible_qubit_idxs)
    qubit_idxs = crbm.visible_qubit_idxs+crbm.hidden_qubit_idxs
    
    visible_idx_map = {visible_qubit_idx:i for i, visible_qubit_idx in enumerate(crbm.visible_qubit_idxs)}
    hidden_idx_map = {hidden_qubit_idx:i for i, hidden_qubit_idx in enumerate(crbm.hidden_qubit_idxs)}
    
    betas = []
    deltas = []
    
    for beta in range(beta_min, beta_max):
        beta = float(beta)
        dwave_weights, dwave_vbias, dwave_hbias = -ising_weights/beta, -ising_vbias/beta, -ising_hbias/beta
        dwave_weights_np = dwave_weights.detach().cpu().numpy()
        biases = torch.cat((dwave_vbias, dwave_hbias)).detach().cpu().numpy()
        
        print("beta = {0} : J range = ({1}, {2}), h range = ({3}, {4})".format(beta, np.min(dwave_weights_np),
                                                                               np.max(dwave_weights_np), np.min(biases),
                                                                               np.max(biases)))
        
        h = {qubit_idx:bias for qubit_idx, bias in zip(qubit_idxs, biases)}
        J = {}
        for edge in crbm_edgelist:
            if edge[0] in crbm.visible_qubit_idxs:
                J[edge] = dwave_weights_np[visible_idx_map[edge[0]]][hidden_idx_map[edge[1]]]
            else:
                J[edge] = dwave_weights_np[visible_idx_map[edge[1]]][hidden_idx_map[edge[0]]]
                    
        response = dwave_sampler.sample_ising(h, J, num_reads=n_reads, auto_scale=False)
        dwave_samples, dwave_energies = batch_dwave_samples(response)
        dwave_energy_exp = np.mean(dwave_energies)
        
        crbm_vis, crbm_hid = crbm_sampler.block_gibbs_sampling()
        
        crbm_vis = torch.where(crbm_vis == ZERO, MINUS_ONE, crbm_vis)
        crbm_hid = torch.where(crbm_hid == ZERO, MINUS_ONE, crbm_hid)
        
        crbm_energy_exp = sample_energies_qpu(dwave_weights, dwave_vbias, dwave_hbias, crbm_vis, crbm_hid)
        crbm_energy_exp = torch.mean(crbm_energy_exp, axis=0)
        
        print("crbm_energy_exp : {0}, dwave_energy_exp : {1}".format(crbm_energy_exp, dwave_energy_exp))
        delta = float(crbm_energy_exp)-float(dwave_energy_exp)
        
        betas.append(beta)
        deltas.append(delta)
        
    return betas, deltas

In [ ]:
betas, detlas = energy_deltas_ising(beta_min=3, beta_max=100, crbm_sampler=model.sampler, dwave_sampler=qpu_sampler)

In [ ]:
deltas = detlas

In [ ]:
def plot_beta_vs_delta(betas, deltas):
    """
    Plot the estimates of beta during the beta estimation procedure   
    """
    fig, ax = plt.subplots(figsize=(40, 16))
    
    plt.plot(betas, deltas)
    
    ax.set_xlabel("Beta", fontsize=60)
    ax.set_ylabel(r"$E_{p_{RBM}}[H_{h, J}] - E_{p_{HW}}[H_{h, J}]$", fontsize=60)
    
    ax.tick_params(axis='both', which='major', labelsize=60)
    ax.grid("True", axis='both', which='both')
    
    plt.show()
    plt.close()

In [ ]:
plot_beta_vs_delta(betas, deltas)

In [ ]:
\qubit_idxs = crbm.visible_qubit_idxs+crbm.hidden_qubit_idxs
for idx in qubit_idxs:
    if idx not in qpu_sampler.nodelist:
        print(idx)

In [ ]:
len(qubit_idxs)

In [ ]:
visible_idx_map_inv = {i:visible_qubit_idx for i, visible_qubit_idx in enumerate(crbm.visible_qubit_idxs)}
hidden_idx_map_inv = {i:hidden_qubit_idx for i, hidden_qubit_idx in enumerate(crbm.hidden_qubit_idxs)}

In [ ]:
rbm_weights_np = crbm_weights.detach().cpu().numpy()

for row in range(rbm_weights_np.shape[0]):
    for col in range(rbm_weights_np.shape[1]):
        if rbm_weights_np[row][col] > 1e-5:
            edge = (visible_idx_map_inv[row], hidden_idx_map_inv[col])
            if edge not in qpu_sampler.edgelist and (edge[1], edge[0]) not in qpu_sampler.edgelist:
                print(edge)

In [ ]:
from models.rbm.qimeraRBM import QimeraRBM
_NUM_NODES = 1000
qrbm = QimeraRBM(n_visible=_NUM_NODES, n_hidden=_NUM_NODES)

In [ ]:
qpu_sampler = DWaveSampler(solver={"topology__type":"chimera", "chip_id":"DW_2000Q_5"})

In [ ]:
qpu_sampler.properties.keys()

In [ ]:
qpu_sampler.properties['chip_id']